In [1]:
# Setup
!pip install --quiet --upgrade pip

# Install the dependencies.
!pip install --quiet -r gdal_req.txt

# Restart the runtime by ending the process.
exit()

In [1]:
import numpy as np
import rasterio
from google.cloud import storage
import os
import matplotlib.pyplot as plt
from serving.constants import  BUCKET, IMG_SOURCE_PREFIX, HIST_DEST_PREFIX, NUM_BANDS, HIST_BINS_LIST, SCALE, CROP
from serving.hist_data import recombine_image, write_histogram_to_gcs
from serving.common import list_blobs_with_prefix
import io
import google.auth
from rasterio.io import MemoryFile
from osgeo import gdal
import time
import logging

In [2]:
# SETUP
credentials, _ = google.auth.default()

bucket_name = BUCKET

directory_prefix = IMG_SOURCE_PREFIX
output_prefix = HIST_DEST_PREFIX

logging.basicConfig(filename="hist.log",level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

counties = ["Lancaster", "Crawford", "Orleans", "Tulare", "Wharton", "Story", "Canyon", "Kit Carson"]
years = range(2017,2019)
months = [5,7,9]

In [3]:
get_input_img_params = [{"county": county,
                        "year": year,
                        "month": month} for county in counties
                                       for year in years
                                       for month in months
                       ]
def img_name_composer(county, year, month):
    return f"{IMG_SOURCE_PREFIX}{county}_{year}_{month}-{month+1}"

images_to_process = [img_name_composer(**params) for params in get_input_img_params]
blob_names = [f"{HIST_DEST_PREFIX}{county}/{year}/{month}-{month+1}/{SCALE}/hist" for county in counties
                                       for year in years
                                       for month in months
                       ]
print(len(blob_names))

48


In [6]:
# Usage
for image_name, blob_name in zip(images_to_process, blob_names):
    recombine_image_hist = recombine_image(BUCKET, image_name, HIST_BINS_LIST, NUM_BANDS)
    write_histogram_to_gcs(recombine_image_hist, BUCKET, blob_name)